In [83]:
import numpy as np
import scipy as sp
from scipy import ndimage
from scipy import stats
import pandas
import copy
import time
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import cv2
from mpl_toolkits.mplot3d import Axes3D
from scipy.linalg import norm
import pylab as plt

#from mpl_toolkits import mplot3d
%matplotlib auto

from mpl_toolkits.mplot3d import Axes3D
#import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation

n_tic = time.process_time()
# img = cv2.imread( r'm31_24proj.png', 0)
# x,y = [], []
# for i in range(277):          #select pixels based on intensity, over range of image
#     for j in range(306):
#         if img[i, j] >= 75:
#             x.append(i)
#             y.append(j)
# xx, yy=[], []
# for i in x:
#     i=-(i-138.5)
#     xx.append(i)
# for j in y:
#     j=j-153
#     yy.append(y)
# IR=np.vstack((yy,xx))

# Y=IR[1]
# X=IR[0]
# inject = np.vstack([X, Y])
# IR = np.append(CR, inject, axis=1)
# Basic cmap b from ds9
cdict= {'red':  ((0., 0.25, 0.25 ),
                 (0.25, 0, 0 ),
                 (0.5, 1, 1),
                 (1, 1, 1)),
 
        'green': ((0., 0, 0 ),
                 (0.5, 0, 0 ),
                 (0.75, 1, 1),
                 (1, 1, 1)),
 
        'blue': ((0, 0.25, 0.25),
                 (0.25, 1, 1),
                 (0.5, 0, 0),
                 (0.75, 0, 0),
                 (1, 1, 1)),
                 }

b_ds9 = LinearSegmentedColormap('b_ds9', cdict)

def build3d(num, theme='dark', grid=True, panel=0.0,boxsize=250, figsize=(8,6), dpi=150):
    ''' 
        Sets basic parameters for 3d plotting 
        and returns the figure and axis object.
        
        Inputs
        -----------
        theme: When set to 'dark' uses a black background
               any other setting will produce a typical white ackground
               
        grid: When true, includes the grid and ticks colored opposite of the background
        
        panel: 0-1, sets the opacity of the grid panels
        
        boxsize: Determines the -/+ boundaries of each axis
        
        figsize, dpi: matplotlib figure parameters
        
    '''
    
    fig = plt.figure(num = num, figsize=figsize, dpi=dpi)
    ax = Axes3D(fig)
    
    if(grid and theme=='dark'):
        color='white'
    else:
        color='black'
    
    if not(grid):
        ax.grid(False)
        
    ax.set_zlim3d([-boxsize, boxsize])
    ax.set_zlabel('Z', color=color)
    ax.set_ylim3d([-boxsize, boxsize])
    ax.set_ylabel('Y', color=color)
    ax.set_xlim3d([-boxsize, boxsize])
    ax.set_xlabel('X', color=color)
    ax.tick_params(axis='both', colors=color)
    
    # Sets pane color transparent
    ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, panel))
    ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, panel))
    ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, panel))

    # For all black black, no tick
    if(theme == 'dark'):
        fig.set_facecolor('black')
        ax.set_facecolor('black')
    
    return fig, ax

def reset3d(ax, theme='dark', grid=False, panel=0.0,boxsize=250, figsize=(8,6), dpi=150):
    ''' 
        resets basic parameters for 3d plotting 
        but returns only modified axis.
        
        Inputs
        -----------
        axis: current axis
        
        theme: When set to 'dark' uses a black background
               any other setting will produce a typical white ackground
               
        grid: When true, includes the grid and ticks colored opposite of the background
        
        panel: 0-1, sets the opacity of the grid panels
        
        boxsize: Determines the -/+ boundaries of each axis
        
    '''

    
    if(grid and theme=='dark'):
        color='white'
    else:
        color='black'
    
    if not(grid):
        ax.grid(False)
        
    ax.set_zlim3d([-boxsize, boxsize])
    ax.set_zlabel('Z', color=color)
    ax.set_ylim3d([-boxsize, boxsize])
    ax.set_ylabel('Y', color=color)
    ax.set_xlim3d([-boxsize, boxsize])
    ax.set_xlabel('X', color=color)
    ax.tick_params(axis='both', colors=color)
    
    # Sets pane color transparent
    ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, panel))
    ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, panel))
    ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, panel))

    # For all black black, no tick
    if(theme == 'dark'):
        ax.set_facecolor('black')
    
    return ax

def get_color(coordinates, kde=True, color='b'):
    ''' Calculates the KDE for the coordinates array
        If all particles leave the boundary, (ie empty coordinate array)
        a single color is returned
        
        Note KDE calculation is a bottleneck, 
        so avoid for large N particles by setting kde==False
    '''
    if(coordinates.shape[1]!=0 and kde):
        kde = stats.gaussian_kde(coordinates)
        color = kde(coordinates)
    
    else:
        color = color
    
    return color

############################################## Spawn Definitions #################################################


def spawn_sphere(CR, particles, rmax ,x0, y0, z0, shell=False):
    ''' 
    generates a spherically uniform distribuiton of particles 
    and append them to the provided CR array
        
    Inputs
    --------
    CR: the array to add particles to (TODO make condition so if CR=None, create sphere)
        
    particles: integer number of particles to produce
        
    rmax: max radius of sphere 
        
    x0,y0,z0: intital coordinates
        
    shell: if True, produces points on a spherical shell instead of solid sphere

    Return
    ---------
    the new particle coordinates array
    '''
    
    phi = np.random.uniform(0, 2*np.pi, particles)
    theta = np.arccos(np.random.uniform(-1, 1, particles))
    
    if shell==True:
        r=rmax
    else:
        r = rmax*pow(np.random.uniform(0, 1, particles), 1./3.)
    X = r*np.cos(phi)*np.sin(theta) + x0
    Y = r*np.sin(phi)*np.sin(theta) + y0
    Z = r*np.cos(theta) + z0
    
    inject = np.vstack([X,Y,Z])
    CR = np.append(CR, inject, axis=1)

    return CR

def spawn_sphere_ring(CR, particles, rmin, rmax ,x0, y0, z0, shell=False):
    ''' 
    generates a spherically uniform distribuiton of particles 
    and append them to the provided CR array
        
    Inputs
    --------
    CR: the array to add particles to (TODO make condition so if CR=None, create sphere)
        
    particles: integer number of particles to produce
        
    rmax: max radius of sphere 
        
    x0,y0,z0: intital coordinates
        
    shell: if True, produces points on a spherical shell instead of solid sphere

    Return
    ---------
    the new particle coordinates array
    '''
    
    phi = np.random.uniform(0, 2*np.pi, particles)
    theta = np.arccos(np.random.uniform(-1, 1, particles))
    
    if shell==True:
        r=rmax
    else:
        r = rmax*np.sqrt(np.random.uniform( pow(rmin/rmax, 2), 1, particles))
    X = r*np.cos(phi)*np.sin(theta) + x0
    Y = r*np.sin(phi)*np.sin(theta) + y0
    Z = r*np.cos(theta) + z0
    
    inject = np.vstack([X,Y,Z])
    CR = np.append(CR, inject, axis=1)

    return CR


def spawn_ring(CR, particles=10, rmin=15, rmax=15, thickness=10, x0=0,y0=0,z0=0, shell=False):
    ''' generates an annular uniform distribuiton of particles 
        and appends them to the provided CR array
        
        Inputs
        --------
        CR: the array to add particles to (TODO make condition so if CR=None, create sphere)
        
        particles: integer number of particles to produce
        
        rmin, rmax: min/max radius of the ring (note, rmin=0 produces a cylinder)
        
        x0,y0,z0: intital coordinates
        
        Return
        ---------
        the new particle coordinates array
    '''
# Note that r here means the radius of the cylinder of the spawn ring
    phi = np.random.uniform(0, 2*np.pi, particles)
    if shell==True:
        r = rmax
    else:
        r = rmax*np.sqrt(np.random.uniform( (pow(rmin/rmax, 2)), 1, particles))
    X = r*np.cos(phi) + x0
    Y = r*np.sin(phi) + y0
    Z = thickness * np.random.uniform(-1, 1, particles) + z0
    
    inject = np.vstack([X,Y,Z])

    CR = np.append(CR, inject, axis=1)
    return CR


def spawn_IR(CR, particles=21613, x0=0, y0=0):
    img = cv2.imread( r'm31_24proj.png', 0)
    X,Y = [], []

    for i in range(277):          #select pixels based on intensity, over range of image
        for j in range(306):
            if img[i, j] >= 75: #75 is intensity
                y=-(i-138.5)# image is flipped wrt y axis so we need to multiply by negative one. The 138.5 comes from needing to center the image at the origin
                x=j-153 #centers image at center
                X.append(x)
                Y.append(y)

            if img[i, j] >= 100:
                y=-(i-138.5)
                x=j-153
                X.append(x)
                Y.append(y)

            if img[i, j] >= 150: 
                y=-(i-138.5)
                x=j-153
                X.append(x)
                Y.append(y)

            if img[i, j] >= 200:
                y=-(i-138.5)
                x=j-153
                X.append(x)
                Y.append(y)


    IR=np.vstack((X,Y))
    X=IR[0]
    Y=IR[1]
    Z=np.random.uniform(0, 1,  len(IR[0])) #Since we don't have z axis just put them random for now
    inject = np.vstack([X, Y, Z])
    CR = np.append(CR, inject, axis=1)
                                   
    return CR

def spawn_H(CR, particles=13955, x0=0, y0=0):
    og_img = plt.imread(r'm31_HIproj.png')
    #Load Image in greyscale using OpenCV:
    img = cv2.imread( r'm31_HIproj.png', 0)
    img_dim = img.shape
    
    X, Y = [], []


    for i in range(img_dim[0]):     #select pixels based on intensity, over x,y range of image
        for j in range(img_dim[1]):
            if img[i, j] >= 125:
                y=-(i-(img_dim[0]*0.5)) #Center image
                x=j-(img_dim[1]*0.5)
                X.append(x)
                Y.append(y)

            if img[i, j] >= 175:
                y=-(i-(img_dim[0]*0.5))
                x=j-(img_dim[1]*0.5)
                X.append(x)
                Y.append(y)

            if img[i, j] >= 200:
                y=-(i-(img_dim[0]*0.5))
                x=j-(img_dim[1]*0.5)
                X.append(x)
                Y.append(y)

            if img[i, j] >= 250:
                y=-(i-(img_dim[0]*0.5))
                x=j-(img_dim[1]*0.5)
                X.append(x)
                Y.append(y)
                
    H=np.vstack((X,Y))
    Y=H[1]
    X=H[0]
    Z=np.random.uniform(0, 1,  len(H[0])) #Since we don't have z axis just put them random for now
    inject = np.vstack([X, Y, Z])
    CR = np.append(CR, inject, axis=1)
    
    return CR
    

####################################################### CR Position definitions #######################################

def initial_CR(particles=100, kde=False):
    ''' Sets the initial particles to be injected
        
        Inputs
        --------
        kde: use kde as density (kde==True) or solid color (kde==False)
        
        Outputs
        --------
        CR, CR_esc, and density arrays
        
        TODO: give acess to initial parameters, 
        either through class or function args
    
    '''
    
    
    CR = np.zeros((3,particles))

    # Samples a uniformly distributed Cylinder
    ''' max_z0 = 10
    max_r0 = 15
    phi = np.random.uniform(0,2*np.pi, particles)
    r = rmax*np.sqrt(np.random.uniform(0, 1, particles))
    X0 = r*np.cos(phi)
    Y0 = r*np.sin(phi)
    Z0 = np.random.uniform(-max_z0, max_z0, particles)
    '''
    # uniform sphere
    # CR = np.random.normal(0,spread, (pieces, 3))
    rmax = 15
    phi = np.random.uniform(0, 2*np.pi, particles)
    theta = np.arccos(np.random.uniform(-1, 1, particles))
    r = rmax*pow(np.random.uniform(0, 1, particles), 1./3.)
    X0 = r*np.cos(phi)*np.sin(theta)
    Y0 = r*np.sin(phi)*np.sin(theta)
    Z0 = r*np.cos(theta)
    
    
    CR[0] = X0
    CR[1] = Y0
    CR[2] = Z0

    # For Normal spherical Gaussian
    # spread = 15
    # CR = np.random.normal(0,spread, (pieces, 3))
    
    CR_esc = np.empty((3,0))    
    density = get_color(CR, kde)
    
    return CR, CR_esc, density

def run_step(CR, CR_esc, rstep, zstep):
    ''' perform one step iteration on the CR density
    
        Inputs
        -------
        CR: array of confined particle coordinates
        CR_esc: array of current escaped particles
        rstep,
        zstep: callable functions for the r and z steps respectively
                      currently, these are the maximum values to draw from 
                      a uniform distribution.
        Outputs
        --------
        updated arrays CR, r, z, CR_esc
    
    '''

    r = np.sqrt(CR[0]**2 + CR[1]**2) 
    z = CR[2]
    
    particles = CR.shape[1]
    
    #r_stepsize = rstep(r,z)
    #r_stepsize = rstep(CR[0],CR[1])
    #z_stepsize = zstep(z,r)
    
    #r_step = np.random.uniform(0, r_stepsize, particles)
    #phi = np.random.uniform(0,2*np.pi, particles)
    
    #Xstep = r_step*np.cos(phi)
    #Ystep = r_step*np.sin(phi)
    #Zstep = np.random.uniform(-z_stepsize, z_stepsize, particles)
            #z_stepsize*np.random.choice([-1,1], size=z.shape)
    
    ################### In progress ##############
    r_stepsize = rstep(CR[0],CR[1], CR[2])
    r_step = np.random.uniform(0, r_stepsize, particles)
    phi = np.random.uniform(0,2*np.pi, particles)
    theta = np.arccos(np.random.uniform(-1, 1, particles))
    
    Xstep = r_step*np.cos(phi)*np.sin(theta)
    Ystep = r_step*np.sin(phi)*np.sin(theta)
    Zstep = .5*r_step*np.cos(theta) #.1 is just to make it so the particle diffusion forms a more disk like shape (obviously just a working parameter for now. still looking for good paper describing step size in z direction)
            #z_stepsize*np.random.choice([-1,1], size=z.shape)
    
    ###############################################

    CR[0] += Xstep
    CR[1] += Ystep
    CR[2] += Zstep
    
    r_free = r > 300 #boundary limits
    z_free  = abs(z) > 200
    
    iter_CR_esc = CR.T[np.logical_or(r_free, z_free )].T
    CR = CR.T[np.logical_not(np.logical_or(r_free, z_free))].T

    CR_esc = np.append(CR_esc, iter_CR_esc, axis=1)
    
    r = np.sqrt(CR[0]**2 + CR[1]**2) 
    z = CR[2]
    

    return CR, r, z, CR_esc, r_step

def step_size(x,y, z):
    x0 = 0
    y0 = 0
    z0 = 0
    rmax = 135
    r = np.sqrt((x-x0)**2 + (y-y0)**2 + (z-z0)**2) #inefficient diffusion bubble
    rmask = (r<rmax).astype(int) #.astype() turns numbers in to ones and zeros
    
    x1 = -155
    y1 = 165
    z1 = 0
    rmax = 35
    r = np.sqrt((x-x1)**2 + (y-y1)**2 + (z-z1)**2) #inefficient diffusion bubble
    rmask1 = (r<rmax).astype(int) #r<rmax since in return it is being subtracted
    
    x2 = 0
    y2 = 0
    z2 = 0
    rmax2 = 20 
    zmax2  = 10
    r2 = np.sqrt((x-x2)**2 + (y-y2)**2)
    Z2 = abs(z-z2)
    cylindrical_halo = (np.logical_and(r2<rmax2, Z2<zmax2 )).astype(int)
    
    x3 = 0
    y3 = 0
    z3 =0
    rmax3 = 100
    zmax = 1
    r3 = np.sqrt((x-x3)**2 + (y-y3)**2 + (z-z3)**2) #inefficient diffusion bubble 
    spherical_halo = (r3<rmax3).astype(int) #r<rmax since in return it is being subtracted
    
    v = 3*10**10 #cm/s^2
    D = 3*10**28 #cm^2/s
    Lambda = (3*D/v)/(3.086*10**21) #average step size for inside the galaxy in kpc


    return 100*(1 - 0*(1-(100*Lambda*spherical_halo)) - 0*(1-(Lambda*cylindrical_halo + (1-(100*Lambda*spherical_halo)))))


# def save_steps(label='gal_CR', kde=True):
#     ''' Runs the diffusion with default parameters
#         and saves some iterations as png images
        
#         Inputs
#         -------
#         label: string that form the output image file name
#                in the form label + iteration +.png
               
#         kde: whether to use kde as density (kde==True) or solid color (kde==False)
        
#         Outputs
#         -------
#         CR, CR_esc, and density arrays
        
#         TODO: Make it easier to change parameters
        
#     '''
    
#     fig, ax = build3d(grid=True, panel=0.3, boxsize=175)
    
#     rstep = step_size
#     zstep = lambda z,r : 0.5

#     CR, CR_esc, density = initial_CR(particles=0)
#     CR = spawn_ring(CR, particles=1000,rmax=65, rmin=55, x0=0, y0=0, z0=0)
#     CR = spawn_sphere(CR, particles=500,rmax=15, x0=0, y0=0, z0=0)

#     density = get_color(CR, kde=True)
#     ax.set_title('M31 - CR Diffusion', color='white')
#     ax.scatter( CR[0],CR[1],CR[2], zdir='z', c=density,cmap='viridis',s=1, alpha=0.75)
#     ax.scatter( CR_esc[0],CR_esc[1],CR_esc[2], zdir='z', c='r',s=1, alpha=0.5)
#     plt.savefig(label + '0.png')
#     for i in range(0,10001):
#         CR, r,z, CR_esc = run_step(CR, CR_esc, rstep, zstep)
#         if(i%10==0):
#             CR = spawn_ring(CR, particles=10,rmax=65, rmin=55, x0=0, y0=0, z0=0)
#             CR = spawn_sphere(CR, particles=10,rmax=10, x0=0, y0=0, z0=0)
#             CR = spawn_sphere(CR, particles=10, rmax=5, x0=55, y0=55, z0=0, shell=True)


#         if (i%100==0):
#             ax.clear()
#             ax = reset3d(ax, grid=True, panel=0.3, boxsize=175)
#             ax.text(75,-70,-70, 'iter: ' + str(i+1), color='white')
#             ax.set_title("M31 - CR Diffusion", color='white')
            
#             density = get_color(CR, kde)
            
#             ax.scatter( CR[0],CR[1],CR[2], zdir='z', c=density,cmap='viridis',s=1, alpha=0.75)
#             ax.scatter( CR_esc[0],CR_esc[1],CR_esc[2], zdir='z', c='r',s=1, alpha=0.5)
#             plt.savefig(label + str(i+1)+'.png')
        
#     return CR, CR_esc, density


##################################### Electron Energy Definitions ################################################################

def e_spawn_sphere(e, particles, rmax ,x0, y0, z0, shell=False):
    ''' 
    generates a spherically uniform distribuiton of particles 
    and append them to the provided CR array
        
    Inputs
    --------
    CR: the array to add particles to (TODO make condition so if CR=None, create sphere)
        
    particles: integer number of particles to produce
        
    rmax: max radius of sphere 
        
    x0,y0,z0: intital coordinates
        
    shell: if True, produces points on a spherical shell instead of solid sphere

    Return
    ---------
    the new particle coordinates array
    '''
    
    phi = np.random.uniform(0, 2*np.pi, particles)
    theta = np.arccos(np.random.uniform(-1, 1, particles))
    
    if shell==True:
        r=rmax
    else:
        r = rmax*pow(np.random.uniform(0, 1, particles), 1./3.)
    X = r*np.cos(phi)*np.sin(theta) + x0
    Y = r*np.sin(phi)*np.sin(theta) + y0
    Z = r*np.cos(theta) + z0
    
    e_initial_NRG = 1000*np.ones(particles)

    inject = np.vstack([X,Y,Z, e_initial_NRG])
    e = np.append(e, inject, axis=1)

    return e


def initial_e(particles=100, kde=False):
    ''' Sets the initial particles to be injected
        
        Inputs
        --------
        kde: use kde as density (kde==True) or solid color (kde==False)
        
        Outputs
        --------
        CR, CR_esc, and density arrays
        
        TODO: give acess to initial parameters, 
        either through class or function args
    
    '''
    
    
    e = np.zeros((4,particles))
    # Samples a uniformly distributed Cylinder
    ''' max_z0 = 10
    max_r0 = 15
    phi = np.random.uniform(0,2*np.pi, particles)
    r = rmax*np.sqrt(np.random.uniform(0, 1, particles))
    X0 = r*np.cos(phi)
    Y0 = r*np.sin(phi)
    Z0 = np.random.uniform(-max_z0, max_z0, particles)
    '''
    # uniform sphere
    # CR = np.random.normal(0,spread, (pieces, 3))
    rmax = 15
    phi = np.random.uniform(0, 2*np.pi, particles)
    theta = np.arccos(np.random.uniform(-1, 1, particles))
    r = rmax*pow(np.random.uniform(0, 1, particles), 1./3.)
    X0 = r*np.cos(phi)*np.sin(theta)
    Y0 = r*np.sin(phi)*np.sin(theta)
    Z0 = r*np.cos(theta)
    
    
    e[0] += X0
    e[1] += Y0
    e[2] += Z0
   
    # For Normal spherical Gaussian
    # spread = 15
    # CR = np.random.normal(0,spread, (pieces, 3))
    
    e_esc = np.empty((4,0)) 
    # e_esc_NRG = np.empty((3,0))
    density = get_color(e, kde)
    
    return e, e_esc, density

def e_run_step(e, e_esc, rstep, zstep):

    delta = 1/3
    e_initial_NRG = e[3]
    r = np.sqrt(e[0]**2 + e[1]**2) 
    z = e[2]

    particles = e.shape[1]
    
    r_stepsize = rstep(e[0],e[1], e[2])
    r_step = np.random.uniform(0, r_stepsize, particles)
    phi = np.random.uniform(0,2*np.pi, particles)
    theta = np.arccos(np.random.uniform(-1, 1, particles))
    
    e_final_NRG = e_initial_NRG/(1+(10**(-5))*r_step*e_initial_NRG)  #energy loss equation for electorns in kpc
    e_initial_NRG = e_final_NRG #change it to back to initial energy for when we run the for loop
    r_step = r_step*(e_initial_NRG**(delta))
    
    Xstep = r_step*np.cos(phi)*np.sin(theta) 
    Ystep = r_step*np.sin(phi)*np.sin(theta)
    Zstep = .5*r_step*np.cos(theta) #.4 is just to make it so the particle diffusion forms a more disk like shape (obviously just a working parameter for now. still looking for good paper describing step size in z direction)

    e[0] += Xstep
    e[1] += Ystep
    e[2] += Zstep
    e[3] = e_initial_NRG

    r_free = r > 300 #boundary limits
    z_free  = abs(z) > 200
    e_min_NRG = e_final_NRG < 1 #minimum energy allowed for a particle

    iter_e_esc = e.T[np.logical_or(e_min_NRG, np.logical_or(r_free, z_free))].T
    e = e.T[np.logical_not(np.logical_or(e_min_NRG, np.logical_or(r_free, z_free )))].T

    e_esc = np.append(e_esc, iter_e_esc, axis=1)
    
    # iter_e_esc_NRG = e_initial_NRG.T[e_min_NRG].T
    # e_initial_NRG = e_initial_NRG.T[np.logical_not(e_min_NRG)].T #stops keeping track of particle once it dips below e_min
    
    # e_esc_NRG = np.append(e_esc_NRG, iter_e_esc_NRG, axis = 1)
    
    r = np.sqrt(e[0]**2 + e[1]**2) 
    z = e[2]
    
    
    return e, r, z, e_esc, r_step




################################################ Column Density ##########################################################

def points_in_cylinder(pt1, pt2, r, q):
    ''''    pt1: the center of your first endpoint in your cylinder (need to input a 3d array)
            pt2: the center of your second endpoint in your cylinder (need to input a 3d array)
            r: radius of your cylinder for your line of sight
            q: this is the 3d point you are checking to see if it is inside the cylinder
            returns: if point is inside the volume it returns the tuple (array([0], dtype=int64),) and if it is outside the 
                     volume it returns (array([], dtype=int64),)'''''
    #math for what's below can be checked on https://math.stackexchange.com/questions/3518495/check-if-a-general-point-is-inside-a-given-cylinder
    
    vec = np.subtract(pt2,pt1)
    const = r * np.linalg.norm(vec)
    return np.where(np.dot(np.subtract(q, pt1), vec) >= 0 and np.dot(np.subtract(q, pt2), vec) <= 0 
                    and np.linalg.norm(np.cross(np.subtract(q, pt1), vec)) <= const)[0] #notice the [0] at the end gives us only the list

def truncated_cone(p0, p1, R0, R1, CR):
    
    v = p1 - p0  # vector in direction of axis that'll give us the height of the cone
    h = np.sqrt(v[0]**2 +v[1]**2 + v[2]**2) #height of cone
    mag = norm(v) # find magnitude of vector
    v = v / mag  # unit vector in direction of axis
    not_v = np.array([1, 1, 0]) # make some vector not in the same direction as v
    if (v == not_v).all():
        not_v = np.array([0, 1, 0])
    n1 = np.cross(v, not_v) # make vector perpendicular to v
    # print n1,'\t',norm(n1)
    n1 /= norm(n1)# normalize n1
    n2 = np.cross(v, n1) # make unit vector perpendicular to v and n1
    
    # surface ranges over t from 0 to length of axis and 0 to 2*pi
    n = 80
    t = np.linspace(0, mag, n)
    theta = np.linspace(0, 2 * np.pi, n)
    # use meshgrid to make 2d arrays
    t, theta = np.meshgrid(t, theta)
    R = np.linspace(R0, R1, n)
    # generate coordinates for surface
    X, Y, Z = [p0[i] + v[i] * t + R * np.sin(theta) * n1[i] + R * np.cos(theta) * n2[i] for i in [0, 1, 2]]
    
    z = CR[2]
    permRadius = R1 - (R1 - R0) * (z /h) #boundary of cone
    pointRadius = (np.sqrt(np.add(np.subtract(CR[0], p0[0])**2, np.subtract(CR[1], p0[1])**2)))
#     print(permRadius)
#     print(pointRadius)
#     print(z, h)
    param1 = np.logical_and(z <= h, z >= p1[2]) #note: if cone is facing down on the origin use p1[0], if facing up, use p0[0]
    params = (np.where(np.logical_and(param1, pointRadius<= permRadius), True, False)) #checks to see if it satisfies both the radius and z parameters
#     print(param1)
#     print(params)
    n_CR = sum(params.astype(int)) #Total number of particles in cone
    
    return n_CR, X,Y,Z

def radial_profile(CR, rmax, rmin, bins = 100):
    r = np.sqrt(CR[0]**2 + CR[1]**2)
    r=(r.astype(np.int))
    rad = np.bincount(r)

    n_bins = bins
    bins = np.linspace(0, len(rad), n_bins+1)
    r = np.linspace(rmin,rmax ,n_bins)
    spacing = max(r)/(n_bins-1) #average space between rings

    s = 0 #initial distance from radius
    density = []
    for i in bins:
        i = np.int(i) #just makes sure numbers are whole values in case we input the wrong linspace value
        a = spacing + s #distance to the outer circle
        area = np.pi*(a**2 - s**2) #area between two circles
        if i > 0:
    #         print('between', c, 'and', i)
            n_particles = sum(rad[c:i])
            row = n_particles/area #row as in the greek letter for density, not the row number in the list
            density.append(row)
        s=a #new inner circle radius for next loop
        c=i #holds on to the current i value so that in the next loop you can refer to the previous step
    r_histogram = np.sqrt(CR[0]**2 + CR[1]**2)
    return r, r_histogram, density, bins


#--------------------------------FIT FUNCTIONS-----------------------------------------------------------------------------            
def model_func(t, N_0, t_0): #N_0 is amplitude, t is number of steps, and t_0 is a function of step size and boundary size
    return N_0 * np.exp(-1*t/t_0)

def fit_exp_nonlinear(t, y):
    opt_parms, parm_cov = sp.optimize.curve_fit(model_func, t, y, p0=[initial, t00], maxfev=initial)
    N_0, t_0 = opt_parms
    return N_0, t_0
################################################ Application #############################################################

# nsteps =1000000
# rstep = step_size #see return from step_size def (this line though is only swapping the title of the def)
# zstep = lambda z,r: 1  #not used in spherical coordinate case

# CR, CR_esc, density = initial_CR(particles=0)
# CR = spawn_sphere(CR, particles=10**3, rmax=5.5, x0=0, y0=0, z0=0, shell=False)
# # CR = spawn_sphere_ring(CR, particles = 6666, rmax=117, rmin=5.5, x0=0, y0=0, z0=0, shell=False)
# # CR = spawn_ring(CR, particles=3333, rmax=200, rmin=117, x0=0, y0=0, z0=0, shell=False)
# # CR = spawn_IR(CR)
# # CR = spawn_H(CR)
# initial = CR.shape[1]

# # audrey's code
# particle_array = []
# t_start = []
# left_bound = .9*initial
# right_bound = 0.10*initial
# tt=[]
# cr=[]
# n00 = []
# too = []
# for n in range(0,nsteps):
#     CR, r,z, CR_esc, r_step = run_step(CR, CR_esc, rstep, zstep)
#     t = n #edit this if you want to say something like 1 step = 1/3 seconds ie set t = t/3
#     tt.append(t)
#     cr.append(CR.shape[1])
#     if CR.shape[1]<= left_bound:
#         particle_array = np.append(CR.shape[1], particle_array)
#         t_start = np.append(t, t_start)
#     if CR.shape[1]<=right_bound:
# #         t_start = np.append(t, t_start)
# #         particle_array = np.append(CR.shape[1], particle_array)
#         break
# # for if (t%#1 == #2):, #1 is every number of steps when the if statement activates, #2 is the initial step. 
# #     if(t%100==99):
# #         CR = spawn_ring(CR, particles=0,rmax=65, rmin=55, x0=0, y0=0, z0=0)
# #         CR = spawn_sphere(CR, particles=0, rmax=15, x0=0, y0=0, z0=0)
# #         CR = spawn_sphere(CR, particles=8, rmax=15, x0=55, y0=55, z0=0, shell=True)
#     if CR.shape[1]<=(np.exp(-1))*initial:   #number of steps to transient
#         tt0 = -t/(np.log(CR.shape[1]/initial))
#         n00.append(CR.shape[1])
#         too.append(t)
# #         rate=(initial)*np.exp(-t/t0)
# # #         N0, t0 = fit_exp_nonlinear(t, particle_array[left_index:right_index+1]) #gives optimized variables
# #         print("Analytical t0 =", t0)
# #         print("Number of Seconds to get to Steady State =" , t+1, "seconds")
# #         break


# particle_array = np.flip(np.array(particle_array))
# t_start = (np.flip(np.array(t_start))).astype(int)
# ttt=t_start
# t_start = np.subtract(t_start, t_start[0])
# left_index = np.where(particle_array== max(particle_array))[0][0]
# right_index = np.where(particle_array== min(particle_array))[0][0]
# bound_range = right_index - left_index + 1
# t00 =-too[0]/(np.log(n00[0]/initial))

# t = np.linspace(left_index, right_index, bound_range)
# N_0, t_0 = fit_exp_nonlinear(t, particle_array[left_index:right_index+1]) #gives optimized variables
# fit_y = model_func(t, N_0, t_0) #Note: t must have more than 100 steps to give accurate output



# # # inside = []   
# # # N = 0 
# # # r_cylinder = 100 #radius of desired cross section
# # # L = 1500 #how far out you want to see
# # # V_cylinder = np.pi*(r_cylinder**2)*L
# # # for i in range(0, len(CR[0])):
# # #     in_or_out = points_in_cylinder(np.array([-750,0,0]),np.array([750,0,0]),r_cylinder,
# # #                             np.array([CR[0][i],CR[1][i],CR[2][i]]))
# # #     if np.size(in_or_out)==1: #if its one it means its inside, zero if it's outside
# # #         inside.append(np.size(in_or_out))
# # #         N += 1 #number of particles inside the cross section
# # # n_CR = N/V_cylinder #column density

# # # print('Average Column Density of Cosmic Rays:', n_CR, 'particles/kpc^3')

# # A0 = np.array([0, 0, 750])
# # A1 = np.array([0, 0, 0])
# # n_CR, X,Y,Z = truncated_cone(A0, A1, 0, 200, CR)
# # print(n_CR,'particles in cone')
 
# %matplotlib inline


# plt.figure(1)
# ax1 = plt.subplot(111)
# #plot the remaining particles vs. time
# ax1.plot(t, particle_array[left_index:right_index+1], linewidth=2.0)
# # plt.plot(tt, cr)

# #plot fitted line
# ax1.plot(t, fit_y, color='orange', 
#         label='Fitted Function:\n $y = %0.2f e^{-t/%0.2f}$' % (N_0, t_0), linewidth=3.0)
# # plt.plot(t+ttt[0], fit_y, color='orange', label='Fitted Function:\n $y = %0.2f e^{-t/%0.2f}$' % (N_0, t_0), linewidth=3.0)
# #add plot lables
# left_percentage = left_bound/initial * 100
# right_percentage = right_bound/initial * 100
# plt.title('%.f to %.f percent of remaining particles' % (left_percentage, right_percentage))
# plt.ylabel('number of particles')
# plt.xlabel('time-step')
# plt.legend(loc='best')
# plt.show()

############################################# transient run ############################################################

# nsteps =0
# rstep = step_size #see return from step_size def (this line though is only swapping the title of the def)
# zstep = lambda z,r: 1  #not used in spherical coordinate case

# CR, CR_esc, density = initial_CR(particles=0, kde = False)
# # CR = spawn_sphere(CR, particles=10**4, rmax=5.5, x0=0, y0=0, z0=0, shell=False)
# # CR = spawn_sphere_ring(CR, particles = 6666, rmax=117, rmin=5.5, x0=0, y0=0, z0=0, shell=False)
# # CR = spawn_ring(CR, particles=3333, rmax=200, rmin=117, x0=0, y0=0, z0=0, shell=False)
# CR = spawn_IR(CR)
# # CR = spawn_H(CR)
# initial = CR.shape[1]

# for n in range(0,nsteps): 
#     t = n #number of steps in one second
#     CR, r,z, CR_esc, r_step = run_step(CR, CR_esc, rstep, zstep)
#     t = t #edit this if you want to say something like 1 step = 1/3 seconds ie set t = t/3
# # for if (t%#1 == #2):, #1 is every number of steps when the if statement activates, #2 is the initial step. 
# #     if(t%100==99):
# #         CR = spawn_ring(CR, particles=0,rmax=65, rmin=55, x0=0, y0=0, z0=0)
# #         CR = spawn_sphere(CR, particles=0, rmax=15, x0=0, y0=0, z0=0)
# #         CR = spawn_sphere(CR, particles=8, rmax=15, x0=55, y0=55, z0=0, shell=True)
# #     if t==round(t_0+ttt[0]):   #number of steps to transient
# #         print("t0 =", t_0 + ttt[0])
# #         break
#     if CR.shape[1]<=(np.exp(-1))*initial:   #number of steps to transient
#         t0 = -t/(np.log(CR.shape[1]/initial))
#         rate=(initial)*np.exp(-t/t0)
# #         N0, t0 = fit_exp_nonlinear(t, particle_array[left_index:right_index+1]) #gives optimized variables
#         print("Analytical t0 =", t0)
#         print("Number of Seconds to get to Steady State =" , t+1, "seconds")
#         break
# # [(print("t0 =", -t/(np.log(CR.shape[1]/initial))),break) for t in range(0,nsteps) if (CR.shape[1]<=(np.exp(-1))*initial)]
# # [((CR, r,z, CR_esc, r_step == run_step(CR, CR_esc, rstep, zstep)),print("t0 =", t_0 + ttt[0])) for t in range(0,round(t_0 + ttt[0]))]
        
# escaped = CR_esc.shape[1]
# print('Initial Particles that Escaped Fraction = {:.3f}% or {:} total'.format( (escaped/initial)*100, escaped))
# print('Particles Remaining:', CR.shape[1])
# print('total particles', CR.shape[1]+CR_esc.shape[1])


 
# %matplotlib auto


# density = get_color(CR, kde=True)    
# fig, ax = build3d(num=2, grid=True, panel=0.5, boxsize=350)
# ax.set_title('M31 - CR Diffusion', color='white')

# ax.scatter( CR[0],CR[1],CR[2], zdir='z', c=density,cmap='viridis',s=1, alpha=0.75)
# ax.scatter( CR_esc[0],CR_esc[1],CR_esc[2], zdir='z', c='r',s=1, alpha=0.5)

# plt.show()

############################################# Electron Energy Simulation ###########################################################

# nsteps =10**7
# rstep = step_size #see return from step_size def (this line though is only swapping the title of the def)
# zstep = lambda z,r : 1 #not used in spherical coordinate case

# e, e_esc, density = initial_e(particles=0)
# e = e_spawn_sphere(e, particles=100, rmax=5.5, x0=0, y0=0, z0=0, shell=False)
# e_initial = e.shape[1]


# # c = 9.72*10**(-12) #speed of light in kpc/s
# # b = 10**(-16) #GeV^-1*sec^-1

# for i in range(0,nsteps+1):
#     t = i
#     e, r, z, e_esc, r_step = e_run_step(e, e_esc, rstep, zstep)
#     if e.shape[1]<=(np.exp(-1))*e_initial:  #just setting this condition for now so that the simulation doesn't take too long. will delete later once able to track when the electorns leave
#         e_final_NRG = e[3] 
#         print(e_final_NRG.shape[0])
#         print(e_final_NRG)
#         print("Number of steps to get to Steady State =" , t+1, "seconds")
#         break 

        

# get_ipython().run_line_magic('matplotlib', 'auto')

# density = get_color(e, kde=True)    
# fig, ax = build3d(num=5, grid=True, panel=0.5, boxsize=350)
# ax.set_title('M31 - CR Diffusion', color='white')

# ax.scatter( e[0],e[1],e[2], zdir='z', c=density,cmap='viridis',s=1, alpha=0.75)
# ax.scatter( e_esc[0],e_esc[1],e_esc[2], zdir='z', c='r',s=1, alpha=0.5)

# plt.show()




###########################################  Density Profile (CR) ########################################################


##Cylindrical region  #still in progress
# # inside = []   
# # N = 0 
# # r_cylinder = 100 #radius of desired cross section
# # L = 1500 #how far out you want to see
# # V_cylinder = np.pi*(r_cylinder**2)*L
# # for i in range(0, len(CR[0])):
# #     in_or_out = points_in_cylinder(np.array([-750,0,0]),np.array([750,0,0]),r_cylinder,
# #                             np.array([CR[0][i],CR[1][i],CR[2][i]]))
# #     if np.size(in_or_out)==1: #if its one it means its inside, zero if it's outside
# #         inside.append(np.size(in_or_out))
# #         N += 1 #number of particles inside the cross section
# # n_CR = N/V_cylinder #column density

# # print('Average Column Density of Cosmic Rays:', n_CR, 'particles/kpc^3')


##Cone Reggion #still in progress
# A0 = np.array([0, 0, 750])
# A1 = np.array([0, 0, 0])
# n_CR, X,Y,Z = truncated_cone(A0, A1, 0, 200, CR)
# print(n_CR,'particles in cone')
# fig = plt.figure(2)
# ax = fig.add_subplot(1, 1, 1, projection='3d')
# ax.plot_surface(X, Y, Z, color='b', linewidth=0, antialiased=False)
# ax.scatter(CR[0],CR[1],CR[2], color='r')

# ##2D Circle region
# r, r_histogram, density, bins = radial_profile(CR, rmax = 300, rmin = 0, bins=100)

# #Graph of Density Profile
# plt.figure(3 , figsize=(8,6))
# ax = plt.subplot()
# ax.plot(r, density)
# ax.set_yscale('log')
# # ax.set_xscale('log')
# plt.title('Density Profile')
# plt.xlabel('Radius (kpc)')
# plt.ylabel('Density')

# #histogram showing radial profile

# plt.figure(4, figsize=(8,6))
# plt.hist(r_histogram, bins=bins)
# plt.xlabel('Radius (kpc)')
# plt.ylabel('Number of Particles')
# plt.title('Radial Profile')



n_toc = time.process_time() 
print("\nComputation time = "+str((n_toc - n_tic ))+"seconds") 

Using matplotlib backend: Qt5Agg

Computation time = 0.0seconds


In [84]:
n_tic = time.process_time()
nsteps =10**7
rstep = step_size #see return from step_size def (this line though is only swapping the title of the def)
zstep = lambda z,r: 1  #not used in spherical coordinate case

CR, CR_esc, density = initial_CR(particles=0, kde = False)
CR = spawn_sphere(CR, particles=10**4, rmax=5.5, x0=0, y0=0, z0=0, shell=False)
# CR = spawn_sphere_ring(CR, particles = 6666, rmax=117, rmin=5.5, x0=0, y0=0, z0=0, shell=False)
# CR = spawn_ring(CR, particles=3333, rmax=200, rmin=117, x0=0, y0=0, z0=0, shell=False)
# CR = spawn_IR(CR)
# CR = spawn_H(CR)
initial = CR.shape[1]

for n in range(0,nsteps): 
    t = n #number of steps in one second
    CR, r,z, CR_esc, r_step = run_step(CR, CR_esc, rstep, zstep)
    t = t #edit this if you want to say something like 1 step = 1/3 seconds ie set t = t/3
# for if (t%#1 == #2):, #1 is every number of steps when the if statement activates, #2 is the initial step. 
#     if(t%100==99):
#         CR = spawn_ring(CR, particles=0,rmax=65, rmin=55, x0=0, y0=0, z0=0)
#         CR = spawn_sphere(CR, particles=0, rmax=15, x0=0, y0=0, z0=0)
#         CR = spawn_sphere(CR, particles=8, rmax=15, x0=55, y0=55, z0=0, shell=True)
#     if t==round(t_0+ttt[0]):   #number of steps to transient
#         print("t0 =", t_0 + ttt[0])
#         break
    if CR.shape[1]<=(np.exp(-1))*initial:   #number of steps to transient
        t0 = -t/(np.log(CR.shape[1]/initial))
        rate=(initial)*np.exp(-t/t0)
#         N0, t0 = fit_exp_nonlinear(t, particle_array[left_index:right_index+1]) #gives optimized variables
        print("Analytical t0 =", t0)
        print("Number of Seconds to get to Steady State =" , t+1, "seconds")
        break
        
n_toc = time.process_time() 
print("\nComputation time = "+str((n_toc - n_tic ))+"seconds")

Analytical t0 = 42.70602252913117
Number of Seconds to get to Steady State = 45 seconds

Computation time = 0.1875seconds


In [73]:
def run_step(CR, r,z, CR_esc, r_step):
    ''' perform one step iteration on the CR density
    
        Inputs
        -------
        CR: array of confined particle coordinates
        CR_esc: array of current escaped particles
        rstep,
        zstep: callable functions for the r and z steps respectively
                      currently, these are the maximum values to draw from 
                      a uniform distribution.
        Outputs
        --------
        updated arrays CR, r, z, CR_esc
    
    '''
#     CR = run[0]
#     r = run[1]
#     z = run[2]
#     CR_esc = run[3]
#     r_step = run[4]


    r = np.sqrt(CR[0]**2 + CR[1]**2) 
    z = CR[2]
    
    particles = CR.shape[1]
    
    #r_stepsize = rstep(r,z)
    #r_stepsize = rstep(CR[0],CR[1])
    #z_stepsize = zstep(z,r)
    
    #r_step = np.random.uniform(0, r_stepsize, particles)
    #phi = np.random.uniform(0,2*np.pi, particles)
    
    #Xstep = r_step*np.cos(phi)
    #Ystep = r_step*np.sin(phi)
    #Zstep = np.random.uniform(-z_stepsize, z_stepsize, particles)
            #z_stepsize*np.random.choice([-1,1], size=z.shape)
    
    ################### In progress ##############
    r_stepsize = rstep(CR[0],CR[1], CR[2])
    r_step = np.random.uniform(0, r_stepsize, particles)
    phi = np.random.uniform(0,2*np.pi, particles)
    theta = np.arccos(np.random.uniform(-1, 1, particles))
    
    Xstep = r_step*np.cos(phi)*np.sin(theta)
    Ystep = r_step*np.sin(phi)*np.sin(theta)
    Zstep = .5*r_step*np.cos(theta) #.1 is just to make it so the particle diffusion forms a more disk like shape (obviously just a working parameter for now. still looking for good paper describing step size in z direction)
            #z_stepsize*np.random.choice([-1,1], size=z.shape)
    
    ###############################################

    CR[0] += Xstep
    CR[1] += Ystep
    CR[2] += Zstep
    
    r_free = r > 300 #boundary limits
    z_free  = abs(z) > 200
    
    iter_CR_esc = CR.T[np.logical_or(r_free, z_free )].T
    CR = CR.T[np.logical_not(np.logical_or(r_free, z_free))].T

    CR_esc = np.append(CR_esc, iter_CR_esc, axis=1)
    
    r = np.sqrt(CR[0]**2 + CR[1]**2) 
    z = CR[2]
    
    run = [CR, r, z, CR_esc, r_step]

    yield run

n_tic = time.process_time()
nsteps =10**7
rstep = step_size #see return from step_size def (this line though is only swapping the title of the def)
zstep = lambda z,r: 1  #not used in spherical coordinate case

CR, CR_esc, density = initial_CR(particles=0, kde = False)
CR = spawn_sphere(CR, particles=10**4, rmax=5.5, x0=0, y0=0, z0=0, shell=False)
# CR = spawn_sphere_ring(CR, particles = 6666, rmax=117, rmin=5.5, x0=0, y0=0, z0=0, shell=False)
# CR = spawn_ring(CR, particles=3333, rmax=200, rmin=117, x0=0, y0=0, z0=0, shell=False)
# CR = spawn_IR(CR)
# CR = spawn_H(CR)
initial = CR.shape[1]
run = np.array([CR, r,z, CR_esc, r_step])


for n in range(0,nsteps): 
    t = n #number of steps in one second
    run = run_step(*run)
    print(run)
    t = t #edit this if you want to say something like 1 step = 1/3 seconds ie set t = t/3
# for if (t%#1 == #2):, #1 is every number of steps when the if statement activates, #2 is the initial step. 
#     if(t%100==99):
#         CR = spawn_ring(CR, particles=0,rmax=65, rmin=55, x0=0, y0=0, z0=0)
#         CR = spawn_sphere(CR, particles=0, rmax=15, x0=0, y0=0, z0=0)
#         CR = spawn_sphere(CR, particles=8, rmax=15, x0=55, y0=55, z0=0, shell=True)
#     if t==round(t_0+ttt[0]):   #number of steps to transient
#         print("t0 =", t_0 + ttt[0])
#         break
    if CR.shape[1]<=(np.exp(-1))*initial:   #number of steps to transient
        t0 = -t/(np.log(CR.shape[1]/initial))
        rate=(initial)*np.exp(-t/t0)
#         N0, t0 = fit_exp_nonlinear(t, particle_array[left_index:right_index+1]) #gives optimized variables
        print("Analytical t0 =", t0)
        print("Number of Seconds to get to Steady State =" , t+1, "seconds")
        break
        
n_toc = time.process_time() 
print("\nComputation time = "+str((n_toc - n_tic ))+"seconds")

<generator object run_step at 0x0000012B2026A9C8>


TypeError: run_step() missing 4 required positional arguments: 'r', 'z', 'CR_esc', and 'r_step'

In [74]:
# import memory_profiler
# import memory_profiler
# import memory_profiler
import time
def check_even(numbers):
    even = []
    for num in numbers:
        if num % 2 == 0: 
            even.append(num*num)
            
    return even
if __name__ == '__main__':
#     m1 = memory_profiler.memory_usage()
    t1 = time.process_time()
    cubes = check_even(range(1000000))
    t2 = time.process_time()
#     m2 = memory_profiler.memory_usage()
    time_diff = t2 - t1
#     mem_diff = m2[0] - m1[0]
    print(f"It took {time_diff} Secs to execute this method")


It took 0.21875 Secs to execute this method


In [75]:
import time
def check_even(numbers):
    for num in numbers:
        if num % 2 == 0:
            yield num * num 
    
if __name__ == '__main__':
#     m1 = memory_profiler.memory_usage()
    t1 = time.process_time()
    cubes = check_even(range(1000000))
    t2 = time.process_time()
#     m2 = memory_profiler.memory_usage()
    time_diff = t2 - t1
#     mem_diff = m2[0] - m1[0]
    print(f"It took {time_diff} Secs to execute this method")

It took 0.015625 Secs to execute this method


In [76]:
def bbb(x):
    yield x*x

a=np.array([1,2,3,4,5])
for i in range(5):
    a = bbb(a)
    lst = list(a)
    print(lst[0][1])
    if lst[0][1]==256:
        print(a)
        break


4


TypeError: unsupported operand type(s) for *: 'generator' and 'generator'

In [78]:
q=np.array([[[1,2,3],[4,5,6]], [[7,8,9],[10,11,12]]])
aa=np.sqrt(q[0]**2 + q[1]**2)
a = aa>8
iter_q_esc = q.T[a].T
q = q.T[np.logical_not(a)].T

q_esc = np.append(q_esc, iter_q_esc, axis=1)


def gimme():
    aa=np.sqrt(q[0]**2 + q[1]**2)
    a = aa>8
    iter_q_esc = q.T[a].T
    q = q.T[np.logical_not(a)].T

    q_esc = np.append(q_esc, iter_q_esc, axis=1)
    
    yield q, q_esc

my_array = np.empty(10)
for i, el in enumerate(gimme()): my_array[i] = el
    
print(my_array)

IndexError: boolean index did not match indexed array along dimension 0; dimension is 3 but corresponding boolean dimension is 2

In [85]:
def run_step(CR, CR_esc):
#parameters
#     zstep = lambda z,r: 1  #not used in spherical coordinate case

    x2 = 0
    y2 = 0
    z2 = 0
    rmax2 = 20 
    zmax2  = 10
    
    x3 = 0
    y3 = 0
    z3 =0
    rmax3 = 100
    zmax = 1
    
    v = 3*10**10 #cm/s^2
    D = 3*10**28 #cm^2/s
    Lambda = (3*D/v)/(3.086*10**21) #average step size for inside the galaxy in kpc
    
    
    for i in range(0,10**(10)):   #number of steps to transient 
#         x = CR[0]
#         y =CR[1]
#         z = CR[2]
        
#         x2 = 0
#         y2 = 0
#         z2 = 0
#         rmax2 = 20 
#         zmax2  = 10
        r2 = np.sqrt((CR[0]-x2)**2 + (CR[1]-y2)**2)
        Z2 = abs(CR[2]-z2)
        cylindrical_halo = (np.logical_and(r2<rmax2, Z2<zmax2 )).astype(int)

#         x3 = 0
#         y3 = 0
#         z3 =0
#         rmax3 = 100
#         zmax = 1
        r3 = np.sqrt((CR[0]-x3)**2 + (CR[1]-y3)**2 + (CR[2]-z3)**2) #inefficient diffusion bubble 
        spherical_halo = (r3<rmax3).astype(int) #r<rmax since in return it is being subtracted

#         v = 3*10**10 #cm/s^2
#         D = 3*10**28 #cm^2/s
#         Lambda = (3*D/v)/(3.086*10**21) #average step size for inside the galaxy in kpc


        r_stepsize = 10*(1 - 0*(1-(100*Lambda*spherical_halo)) - 0*(1-(Lambda*cylindrical_halo + (1-(100*Lambda*spherical_halo)))))



        r = np.sqrt(CR[0]**2 + CR[1]**2) 
        z = CR[2]

        particles = CR.shape[1]

        #r_stepsize = rstep(r,z)
        #r_stepsize = rstep(CR[0],CR[1])
        #z_stepsize = zstep(z,r)

        #r_step = np.random.uniform(0, r_stepsize, particles)
        #phi = np.random.uniform(0,2*np.pi, particles)

        #Xstep = r_step*np.cos(phi)
        #Ystep = r_step*np.sin(phi)
        #Zstep = np.random.uniform(-z_stepsize, z_stepsize, particles)
                #z_stepsize*np.random.choice([-1,1], size=z.shape)

        ################### In progress ##############
#         r_stepsize = rstep(CR[0],CR[1], CR[2])
        r_step = np.random.uniform(0, r_stepsize, particles)
        phi = np.random.uniform(0,2*np.pi, particles)
        theta = np.arccos(np.random.uniform(-1, 1, particles))

        Xstep = r_step*np.cos(phi)*np.sin(theta)
        Ystep = r_step*np.sin(phi)*np.sin(theta)
        Zstep = .5*r_step*np.cos(theta) #.1 is just to make it so the particle diffusion forms a more disk like shape (obviously just a working parameter for now. still looking for good paper describing step size in z direction)
                #z_stepsize*np.random.choice([-1,1], size=z.shape)

        ###############################################

        CR[0] += Xstep
        CR[1] += Ystep
        CR[2] += Zstep

        r_free = r > 300 #boundary limits
        z_free  = abs(z) > 200

        iter_CR_esc = CR.T[np.logical_or(r_free, z_free )].T
        CR = CR.T[np.logical_not(np.logical_or(r_free, z_free))].T

        CR_esc = np.append(CR_esc, iter_CR_esc, axis=1)

#         r = np.sqrt(CR[0]**2 + CR[1]**2) 
#         z = CR[2]


#         yield CR, CR_esc
        
        if CR.shape[1]<=(np.exp(-1))*initial:
            return CR, CR_esc
            break
            

            
            
n_tic = time.process_time()

CR, CR_esc, density = initial_CR(particles=0, kde = False)
CR = spawn_sphere(CR, particles=10**3, rmax=5.5, x0=0, y0=0, z0=0, shell=False)
# CR = spawn_sphere_ring(CR, particles = 6666, rmax=117, rmin=5.5, x0=0, y0=0, z0=0, shell=False)
# CR = spawn_ring(CR, particles=3333, rmax=200, rmin=117, x0=0, y0=0, z0=0, shell=False)
# CR = spawn_IR(CR)
# CR = spawn_H(CR)
initial = CR.shape[1]

CR, CR_esc = run_step(CR, CR_esc)
print(len(CR_esc[0]))

# lst = list(run_step(CR, CR_esc))
# CR =lst[0][0]
# CR_esc = lst[0][1]
# print(len(CR_esc[0]))

# escaped = CR_esc.shape[1]
# print('Initial Particles that Escaped Fraction = {:.3f}% or {:} total'.format( (escaped/initial)*100, escaped))
# print('Particles Remaining:', CR.shape[1])
# print('total particles', CR.shape[1]+CR_esc.shape[1])


 
# %matplotlib auto


# density = get_color(CR, kde=True)    
# fig, ax = build3d(num=2, grid=True, panel=0.5, boxsize=350)
# ax.set_title('M31 - CR Diffusion', color='white')

# ax.scatter( CR[0],CR[1],CR[2], zdir='z', c=density,cmap='viridis',s=1, alpha=0.75)
# ax.scatter( CR_esc[0],CR_esc[1],CR_esc[2], zdir='z', c='r',s=1, alpha=0.5)

# plt.show()


n_toc = time.process_time() 
print("\nComputation time = "+str((n_toc - n_tic ))+"seconds")

633

Computation time = 2.265625seconds
